# 0. 正则表达式
- ![](https://tva1.sinaimg.cn/large/006y8mN6gy1g72we30ki3j317j0u0hbn.jpg)

# 1 简单的search和match
- https://tinyurl.com/pike-regexp

- c    matches any literal character c
- .    matches any single character
-     ^    matches the beginning of the input string
-     $    matches the end of the input string
-     *    matches zero or more occurrences of the previous character

In [24]:
def search(pattern,text):
    "return True if the pattern appears anywhere in text"
    if pattern.startswith('^'):
        return match(pattern[1:],text)
    else:
        return match('.*'+pattern,text)
        
def match(pattern,text):
    "return True if the pattern appears at the start of text"
    if pattern=='':
        return True
    elif pattern=="$":
        return text==''
    elif len(pattern)>1 and pattern[1] in '*?':
        p,op,pat = pattern[0],pattern[1],pattern[2:]
        if op=='*':
            return match_star(p,pat,text)
        elif op=='?':
            if match1(p,text) and match(pat,text[1:]):
                return True
            else:
                return match(pat,text)
    else:
        return (match1(pattern[0],text) and match(pattern[1:],text[1:]))

def match1(p,text):
    "return true if first character of text match pattern character p"
    if not text:return False
    return p=='.' or p==text[0]
def match_star(p,pattern,text):
    "return true if any number of char p fllowed by pattern matchs text"
    return (match(pattern,text) or 
            (match1(p,text) and match_star(p,pattern,text[1:])))        

In [46]:
print(match('.?abc','dabcjfd'))

True


# 2. language
- 编译器与解释器区别

![](https://tva1.sinaimg.cn/large/006y8mN6gy1g7ed1favqhj30aw05cdge.jpg)

In [11]:
def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]]) if test else null
    elif 'oneof' == op:
        return set([text[1:]]) if text.startswith(x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)
        
null = frozenset()

def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y
   
def test():
    assert matchset(('lit', 'abc'), 'abcdef')            == set(['def'])
    assert matchset(('seq', ('lit', 'hi '),
                     ('lit', 'there ')), 
                   'hi there nice to meet you')          == set(['nice to meet you'])
    assert matchset(('alt', ('lit', 'dog'), 
                    ('lit', 'cat')), 'dog and cat')      == set([' and cat'])
    assert matchset(('dot',), 'am i missing something?') == set(['m i missing something?'])
    assert matchset(('oneof', 'a'), 'aabc123')           == set(['abc123'])
    assert matchset(('eol',),'')                         == set([''])
    assert matchset(('eol',),'not end of line')          == frozenset([])
    assert matchset(('star', ('lit', 'hey')), 'heyhey!') == set(['!', 'heyhey!', 'hey!'])
    
    return 'tests pass'

print(test())

tests pass


In [13]:
def lit(string):  return ('lit', string)
def seq(x, y):    return ('seq', x, y)
def alt(x, y):    return ('alt', x, y)
def star(x):      return ('star', x)
def plus(x):      return seq(x,star(x))
def opt(x):       return alt(lit(''), x) #opt(x) means that x is optional
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ('eol',)

def test():
    assert lit('abc')         == ('lit', 'abc')
    assert seq(('lit', 'a'), 
               ('lit', 'b'))  == ('seq', ('lit', 'a'), ('lit', 'b'))
    assert alt(('lit', 'a'), 
               ('lit', 'b'))  == ('alt', ('lit', 'a'), ('lit', 'b'))
    assert star(('lit', 'a')) == ('star', ('lit', 'a'))
    assert plus(('lit', 'c')) == ('seq', ('lit', 'c'), 
                                  ('star', ('lit', 'c')))
    assert opt(('lit', 'x'))  == ('alt', ('lit', ''), ('lit', 'x'))
    assert oneof('abc')       == ('oneof', ('a', 'b', 'c'))
    return 'tests pass'

print(test())

tests pass


In [16]:
#---------------
# User Instructions
#
# Complete the search and match functions. Match should
# match a pattern only at the start of the text. Search
# should match anywhere in the text.

def search(pattern, text):
    "Match pattern anywhere in text; return longest earliest match or None."
    for i in range(len(text)):
        m = match(pattern, text[i:])
        if m:
            return m
        
def match(pattern, text):
    "Match pattern against start of text; return longest match found or None."
    remainders = matchset(pattern, text)
    if remainders:
        shortest = min(remainders, key=len)
        return text[:len(text)-len(shortest)]
    
def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y

def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]]) if text else null
    elif 'oneof' == op:
        return set([text[1:]]) if text.startswith(x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)
    
null = frozenset()

def lit(string):  return ('lit', string)
def seq(x, y):    return ('seq', x, y)
def alt(x, y):    return ('alt', x, y)
def star(x):      return ('star', x)
def plus(x):      return seq(x, star(x))
def opt(x):       return alt(lit(''), x)
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ('eol',)

def test():
    assert match(('star', ('lit', 'a')),'aaabcd') == 'aaa'
    assert match(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == None
    assert match(('alt', ('lit', 'b'), ('lit', 'a')), 'ab') == 'a'
    assert search(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == 'b'
    return 'tests pass'

print(test())

tests pass


## 2.1 compiler
- 本课需要编译成python函数即可，low-level
- C语言的Compiler编译成actual machine instructions (实际的机器指令），最高级的编译器
- python和java的编译器（intermediater中级）编译成虚拟机，有一套指令集


In [20]:
# python的编译过程
import dis
dis.dis(lambda x,y:sqrt(x**2+y**2))

  3           0 LOAD_GLOBAL              0 (sqrt)
              2 LOAD_FAST                0 (x)
              4 LOAD_CONST               1 (2)
              6 BINARY_POWER
              8 LOAD_FAST                1 (y)
             10 LOAD_CONST               1 (2)
             12 BINARY_POWER
             14 BINARY_ADD
             16 CALL_FUNCTION            1
             18 RETURN_VALUE


In [27]:
# 用编译器解决之前的问题，编译成python函数
def match(pattern, text):
    "Match pattern against start of text; return longest match found or None."
    remainders = pattern(text)
    if remainders:
        shortest = min(remainders, key=len)
        return text[:len(text)-len(shortest)]
    
def lit(s): return lambda t: set([t[len(s):]]) if t.startswith(s) else null
def seq(x, y): return lambda t: set().union(*map(y, x(t)))
def alt(x, y): return lambda t: x(t) | y(t)
def oneof(chars): return lambda t: set([t[1:]]) if (t and t[0] in chars) else null
dot = lambda t: set([t[1:]]) if t else null
eol = lambda t: set(['']) if t == '' else null
def star(x): return lambda t: (set([t]) | 
                               set(t2 for t1 in x(t) if t1 != t
                                   for t2 in star(x)(t1)))

null = frozenset([])

def test():
    assert match(star(lit('a')), 'aaaaabbbaa') == 'aaaaa'
    assert match(lit('hello'), 'hello how are you?') == 'hello'
    assert match(lit('x'), 'hello how are you?') == None
    assert match(oneof('xyz'), 'x**2 + y**2 = r**2') == 'x'
    assert match(oneof('xyz'), '   x is here!') == None
    return 'tests pass'
print(test())

tests pass


In [28]:
# 增加参数Ns
def lit(s):         return lambda Ns: set([s]) if len(s) in Ns else null
def alt(x, y):      return lambda Ns: x(Ns)|y(Ns)
def star(x):        return lambda Ns: opt(plus(x))(Ns)
def plus(x):        return lambda Ns: genseq(x, star(x), Ns, startx=1) #Tricky
def oneof(chars):   return lambda Ns: set(chars) if 1 in Ns else null
def seq(x, y):      return lambda Ns: genseq(x, y, Ns)
def opt(x):         return alt(epsilon, x)
dot = oneof('?')    # You could expand the alphabet to more chars.
epsilon = lit('')   # The pattern that matches the empty string.

null = frozenset([])

def genseq(x,y,Ns,startx=0):
    "set of matches to xy whose total len is in Ns"
    # Tricky: x+ is defined as:x+=x x*
    # to stop the recursion, the first x must generate at least 1 char
    # and then the recursion x* has that many fewer characters, we use
    # startx=1 to say that x must match at least 1 character
    if not Ns:
        return null
    xmatches = x(set(range(startx,max(Ns)+1)))
    Ns_x = set(len(m) for m in xmatches)
    Ns_y = set(n-m for n in Ns for m in Ns_x if n-m>=0)
    ymatches = y(Ns_y)
    return set(m1+m2
              for m1 in xmatches for m2 in ymatches
              if len(m1+m2) in Ns)
    

def test():
    
    f = lit('hello')
    assert f(set([1, 2, 3, 4, 5])) == set(['hello'])
    assert f(set([1, 2, 3, 4]))    == null 
    
    g = alt(lit('hi'), lit('bye'))
    assert g(set([1, 2, 3, 4, 5, 6])) == set(['bye', 'hi'])
    assert g(set([1, 3, 5])) == set(['bye'])
    
    h = oneof('theseletters')
    assert h(set([1, 2, 3])) == set(['t', 'h', 'e', 's', 'l', 'r'])
    assert h(set([2, 3, 4])) == null
    
    return 'tests pass'
print(test())

tests pass


## 2.2 重构seq，使其能接受多个参数
- 使用decorate 是原本的函数可接受多个参数，不需要改变与这个函数相关的其他任何函数

In [71]:
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        return x if not args else f(x,n_ary_f(*args))
    return n_ary_f

In [72]:
# 将两个数相加的函数变成多个数相加的函数
def add(x,y):
    return x+y
arry_add=n_ary(add)
arry_add(3,6,7,8,9,0,10)

43

In [73]:
# 使用decorate
@n_ary
def add(x,y):
    return x+y
add(3,6,7,8,9,0,10)

43

In [74]:
@n_ary
def seq(x,y): return('seq',x,y)
seq('a','b','c','d','e','f')

('seq', 'a', ('seq', 'b', ('seq', 'c', ('seq', 'd', ('seq', 'e', 'f')))))

In [76]:
# 修复documentation错误
help(seq)

Help on function n_ary_f in module __main__:

n_ary_f(x, *args)



In [3]:
from functools import update_wrapper
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        return x if not args else f(x,n_ary_f(*args))
    update_wrapper(n_ary_f,f)
    return n_ary_f

@n_ary
def seq(x,y): return('seq',x,y)

help(seq)

Help on function seq in module __main__:

seq(x, y)



In [4]:
# 将修复documentation的功能变为decorate，因为DRY
from functools import update_wrapper
def decorator(d):
    "make function d a decorator,d wraps a function fn "
#     def _d(fn):
#         return update_wrapper(d(fn),fn)
#     update_wrapper(_d,d)
#     return _d
    return lambda fn:update_wrapper(d(fn),fn)

@n_ary
def seq(x,y): return('seq',x,y)

help(seq)

Help on function seq in module __main__:

seq(x, y)



## 2.3 cache管理

In [15]:
@decorator
def memo(f):
    """decorator that caches the return value for each call to f(args).
    then when called again with some args, we can just look it up"""
    cache={}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args]=result=f(*args)
            return result
        except TypeError:
            # some element of args can't be a dict key
            return f(args)
    return _f  

In [16]:
callcounts = {}

@decorator
def countcalls(f):
    "Decorator that makes the function count calls to it, in callcounts[f]"
    def _f(*args):
        callcounts[_f]+=1
        return f(*args)
    callcounts[_f] = 0
    return _f


@countcalls
@memo # 不要@memo 2692537  要memo 59次
def fib(n):return 1 if n<=1 else fib(n-1)+fib(n-2)

In [17]:
fib(30)

1346269

In [18]:
callcounts

{<function __main__.fib(n)>: 59}

## 2.4 trace

In [25]:
@decorator
def trace(f):
    indent = '   '
    def _f(*args):
        signature = '%s(%s)' % (f.__name__, ', '.join(map(repr, args)))
        print('%s--> %s' % (trace.level*indent, signature))
        trace.level += 1
        try:
            result = f(*args)
            print('%s<-- %s == %s' % ((trace.level-1)*indent, 
                                      signature, result))
        finally:
            trace.level -= 1
        return result
    trace.level = 0
    return _f

@trace
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

fib(6) #running this in the browser's IDE  will not display the indentations!

--> fib(6)
   --> fib(5)
      --> fib(4)
         --> fib(3)
            --> fib(2)
               --> fib(1)
               <-- fib(1) == 1
               --> fib(0)
               <-- fib(0) == 1
            <-- fib(2) == 2
            --> fib(1)
            <-- fib(1) == 1
         <-- fib(3) == 3
         --> fib(2)
            --> fib(1)
            <-- fib(1) == 1
            --> fib(0)
            <-- fib(0) == 1
         <-- fib(2) == 2
      <-- fib(4) == 5
      --> fib(3)
         --> fib(2)
            --> fib(1)
            <-- fib(1) == 1
            --> fib(0)
            <-- fib(0) == 1
         <-- fib(2) == 2
         --> fib(1)
         <-- fib(1) == 1
      <-- fib(3) == 3
   <-- fib(5) == 8
   --> fib(4)
      --> fib(3)
         --> fib(2)
            --> fib(1)
            <-- fib(1) == 1
            --> fib(0)
            <-- fib(0) == 1
         <-- fib(2) == 2
         --> fib(1)
         <-- fib(1) == 1
      <-- fib(3) == 3
      --> fib(2)
         --> fib

13

### decorator总结
- 用于debug：使用countcalls和trace
- 提升性能：使用 memo
- 扩展函数参数：使用n_ary
- 禁止decorator：disabled

## 2.5 paser

In [36]:
import functools
import re

# 将一个已知的字符串语法规则转为字典
def grammar(description, whitespace = r'\s*'):
    """
    Convert a description to a grammar. Each line is a rule for a
    non-terminal symbol; it looks like this:
        Symbol => A1 A2 ... | B1 B2 ... | C1 C2 ...
    where the right-hand side is one or more alternatives, separated by
    the '|' sign. Each alternative is a sequence of atoms, separated by
    spaces.  An atom is either a symbol on syme left-hand side, or it is a
    regular expression that will be passed to re.match to match a token.
    
    Notation for *, +, or ? not allowed in a rule alternative (but ok within a
    token). Use '\' to continue long lines. You must include spaces or tabs
    around '=>' and '|'. That's within the grammar description itself(...?). The
    grammar that gets defined allows whitespace between tokens by default or
    specify '' as the second argument to grammar() to disallow this (or supply
    any regular expression to describe allowable whitespace between
    tokens)."""
    G = {' ': whitespace}
    description = description.replace('\t', ' ') # no tabs!
    for line in split(description, '\n'):
        lhs, rhs = split(line, ' => ', 1)
        alternatives = split(rhs, ' | ')
        G[lhs] = tuple(map(split, alternatives))
    return G
def split(text, sep = None, maxsplit = -1):
    "Like str.split applied to text, but strips whitespace from each piece."
    return [t.strip() for t in text.strip().split(sep, maxsplit) if t]

G = grammar(r"""
Exp => Term [+-] Exp | Term
Term => Factor [*/] Term | Factor
Factor => Funcall | Var | Num | [(] Exp [)]
Funcall => Var [(] Exps [)]
Exps => Exp [,] Exps | Exp
Var => [a-zA-Z_]\w*
Num => [-+]?[0-9]+([.][0-9]*)?
""")

print(G)

## Parsing URLs
## See http://www.w3.org/Addressing/URL/5_BNF.html

URL = grammar("""
url => httpaddress | ftpaddress | mailtoaddress
httpaddress => http:// hostport /path? ?search?
ftpaddress => ftp:// login / path ; ftptype | ftp:// login / path
/path? => / path | ()
?search? => [?] search | ()
mailtoaddress => mailto: xalphas @ hostname
hostport => host : port | host
host => hostname | hostnumber
hostname => ialpha . hostname | ialpha
hostnumber => digits . digits . digits . digits
ftptype => A formcode | E formcode | I | L digits
formcode => [NTC]
port => digits | path
path => void | segment / path | segment
segment => xalphas
search => xalphas + search | xalphas
login => userpassword hostport | hostport
userpassword => user : password @ | user @
user => alphanum2 user | alphanum2
password => alphanum2 password | password
path => void | segment / path | segment
void => ()
digits => digit digits | digit
digit => [0-9]
alpha => [a-zA-Z]
safe => [-$_@.&+]
extra => [()!*''""]
escape => % hex hex
hex => [0-9a-fA-F]
alphanum => alpha | digit
alphanums => alphanum alphanums | alphanum
alphanum2 => alpha | digit | [-_.+]
ialpha => alpha xalphas | alpha
xalphas => xalpha xalphas | xalpha
xalpha => alpha | digit | safe | extra | escape
""", whitespace = '()')

print(URL)

{' ': '\\s*', 'Exp': (['Term', '[+-]', 'Exp'], ['Term']), 'Term': (['Factor', '[*/]', 'Term'], ['Factor']), 'Factor': (['Funcall'], ['Var'], ['Num'], ['[(]', 'Exp', '[)]']), 'Funcall': (['Var', '[(]', 'Exps', '[)]'],), 'Exps': (['Exp', '[,]', 'Exps'], ['Exp']), 'Var': (['[a-zA-Z_]\\w*'],), 'Num': (['[-+]?[0-9]+([.][0-9]*)?'],)}
{' ': '()', 'url': (['httpaddress'], ['ftpaddress'], ['mailtoaddress']), 'httpaddress': (['http://', 'hostport', '/path?', '?search?'],), 'ftpaddress': (['ftp://', 'login', '/', 'path', ';', 'ftptype'], ['ftp://', 'login', '/', 'path']), '/path?': (['/', 'path'], ['()']), '?search?': (['[?]', 'search'], ['()']), 'mailtoaddress': (['mailto:', 'xalphas', '@', 'hostname'],), 'hostport': (['host', ':', 'port'], ['host']), 'host': (['hostname'], ['hostnumber']), 'hostname': (['ialpha', '.', 'hostname'], ['ialpha']), 'hostnumber': (['digits', '.', 'digits', '.', 'digits', '.', 'digits'],), 'ftptype': (['A', 'formcode'], ['E', 'formcode'], ['I'], ['L', 'digits']), 'for

In [37]:
def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return functools.update_wrapper(d(fn), fn)
    functools.update_wrapper(_d, d)
    return _d

@decorator
def memo(f):
    """Decorator that caches the return value for each call to f(args).
    Then when called again with same args, we can just look it up."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args can't be a dict key
            return f(args)
    return _f

# 按照语法规则解析text
def parse(start_symbol, text, grammar):
    """Example call: parse('Exp', '3*x + b', G).
    Returns a (tree, remainder) pair. If remainder is '', it parsed the whole
    string. Failure if remainder is None. This is a deterministic PEG parser,
    so rule order (left-to-right) matters. Do 'E => T op E | T', putting the
    longest parse first; don't do 'E => T | T op E'
    Also, no left recursion allowed: don't do 'E => E op T'
    See: http://en.wikipedia.org/wiki/Parsing_expression_grammar
    """
    tokenizer = grammar[' '] + '(%s)'

    def parse_sequence(sequence, text):
        """
        Try to match the sequence of atoms against text.
        
        Parameters:
            sequence : an iterable of atoms
            text : a string
        Returns:
            Fail : if any atom in sequence does not match
            (tree, remainder) : the tree and remainder if the entire sequence matches text
        """
        result = []
        for atom in sequence:
            tree, text = parse_atom(atom, text)
            if text is None: return Fail
            result.append(tree)
        return result, text

    @memo
    def parse_atom(atom, text):
        """
        Parameters:
        atom : either a key in grammar or a regular expression
        text : a string
        Returns:
        Fail : if no match can be found
        (tree, remainder) : if a match is found
            tree is the parse tree of the first match found
            remainder is the text that was not matched
        """
        if atom in grammar:  # Non-Terminal: tuple of alternatives
            for alternative in grammar[atom]:
                tree, rem = parse_sequence(alternative, text)
                if rem is not None: return [atom]+tree, rem  
            return Fail
        else:  # Terminal: match characters against start of text
            m = re.match(tokenizer % atom, text)
            return Fail if (not m) else (m.group(1), text[m.end():])
    
    return parse_atom(start_symbol, text)

Fail = (None, None)

def verify(G):
    lhstokens = set(G) - set([' '])
    print(G.values())
    rhstokens = set(t for alts in G.values() for alt in alts for t in alt)
    def show(title, tokens): print(title, '=', ' '.join(map(repr, sorted(tokens))))
    show('Non-Terms', G)
    show('Terminals', rhstokens - lhstokens)
    show('Suspects', [t for t in (rhstokens-lhstokens) if t.isalnum()])
    show('Orphans ', lhstokens-rhstokens)

if __name__ == '__main__':
    verify(G)    
    print(parse('Exp', '3*x + b', G))
    print(parse('Term', '3*x + b', G))

    # print(URL)
    # verify(URL)
    # print(parse('url', 'http://www.w3.org/Addressing/URL/5_BNF.html', URL))

dict_values(['\\s*', (['Term', '[+-]', 'Exp'], ['Term']), (['Factor', '[*/]', 'Term'], ['Factor']), (['Funcall'], ['Var'], ['Num'], ['[(]', 'Exp', '[)]']), (['Var', '[(]', 'Exps', '[)]'],), (['Exp', '[,]', 'Exps'], ['Exp']), (['[a-zA-Z_]\\w*'],), (['[-+]?[0-9]+([.][0-9]*)?'],)])
Non-Terms = ' ' 'Exp' 'Exps' 'Factor' 'Funcall' 'Num' 'Term' 'Var'
Terminals = '*' '[(]' '[)]' '[*/]' '[+-]' '[,]' '[-+]?[0-9]+([.][0-9]*)?' '[a-zA-Z_]\\w*' '\\' 's'
Suspects = 's'
Orphans  = 
(['Exp', ['Term', ['Factor', ['Num', '3']], '*', ['Term', ['Factor', ['Var', 'x']]]], '+', ['Exp', ['Term', ['Factor', ['Var', 'b']]]]], '')
(['Term', ['Factor', ['Num', '3']], '*', ['Term', ['Factor', ['Var', 'x']]]], ' + b')


# 3 作业

## 3.1 json parser

In [41]:
# ---------------
# User Instructions
#
# In this problem, you will be using many of the tools and techniques
# that you developed in unit 3 to write a grammar that will allow
# us to write a parser for the JSON language. 
#
# You will have to visit json.org to see the JSON grammar. It is not 
# presented in the correct format for our grammar function, so you 
# will need to translate it.

# ---------------
# Provided functions
#
# These are all functions that were built in unit 3. They will help
# you as you write the grammar.  Add your code at line 102.

from functools import update_wrapper
# from str import split
import re

def grammar(description, whitespace=r'\s*'):
    """Convert a description to a grammar.  Each line is a rule for a
    non-terminal symbol; it looks like this:
        Symbol =>  A1 A2 ... | B1 B2 ... | C1 C2 ...
    where the right-hand side is one or more alternatives, separated by
    the '|' sign.  Each alternative is a sequence of atoms, separated by
    spaces.  An atom is either a symbol on some left-hand side, or it is
    a regular expression that will be passed to re.match to match a token.
    
    Notation for *, +, or ? not allowed in a rule alternative (but ok
    within a token). Use '\' to continue long lines.  You must include spaces
    or tabs around '=>' and '|'. That's within the grammar description itself.
    The grammar that gets defined allows whitespace between tokens by default;
    specify '' as the second argument to grammar() to disallow this (or supply
    any regular expression to describe allowable whitespace between tokens)."""
    G = {' ': whitespace}
    description = description.replace('\t', ' ') # no tabs!
    for line in split(description, '\n'):
        lhs, rhs = split(line, ' => ', 1)
        alternatives = split(rhs, ' | ')
        G[lhs] = tuple(map(split, alternatives))
    return G

def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return update_wrapper(d(fn), fn)
    update_wrapper(_d, d)
    return _d

@decorator
def memo(f):
    """Decorator that caches the return value for each call to f(args).
    Then when called again with same args, we can just look it up."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args can't be a dict key
            return f(args)
    return _f

def parse(start_symbol, text, grammar):
    """Example call: parse('Exp', '3*x + b', G).
    Returns a (tree, remainder) pair. If remainder is '', it parsed the whole
    string. Failure iff remainder is None. This is a deterministic PEG parser,
    so rule order (left-to-right) matters. Do 'E => T op E | T', putting the
    longest parse first; don't do 'E => T | T op E'
    Also, no left recursion allowed: don't do 'E => E op T'"""

    tokenizer = grammar[' '] + '(%s)'

    def parse_sequence(sequence, text):
        result = []
        for atom in sequence:
            tree, text = parse_atom(atom, text)
            if text is None: return Fail
            result.append(tree)
        return result, text

    @memo
    def parse_atom(atom, text):
        if atom in grammar:  # Non-Terminal: tuple of alternatives
            for alternative in grammar[atom]:
                tree, rem = parse_sequence(alternative, text)
                if rem is not None: return [atom]+tree, rem  
            return Fail
        else:  # Terminal: match characters against start of text
            m = re.match(tokenizer % atom, text)
            return Fail if (not m) else (m.group(1), text[m.end():])
    
    # Body of parse:
    return parse_atom(start_symbol, text)

Fail = (None, None)

JSON = grammar("""object => {} | { members }
members => pair [,] members | pair
pair => string : value
array => [[] []] | [[] elements []]
elements => value , elements | value
value => string | number | object | array | true | false | null
string => "[^"]*"
number => int frac exp | int frac | int exp | int
int => -?[1-9][0-9]*
frac => [.][0-9]+
exp => [eE][-+]?[0-9]+""", whitespace='\s*')

def json_parse(text):
    return parse('value', text, JSON)

def test():
    assert json_parse('["testing", 1, 2, 3]') == (                      
                       ['value', ['array', '[', ['elements', ['value', 
                       ['string', '"testing"']], ',', ['elements', ['value', ['number', 
                       ['int', '1']]], ',', ['elements', ['value', ['number', 
                       ['int', '2']]], ',', ['elements', ['value', ['number', 
                       ['int', '3']]]]]]], ']']], '')
    
    assert json_parse('-123.456e+789') == (
                       ['value', ['number', ['int', '-123'], ['frac', '.456'], ['exp', 'e+789']]], '')
    
    assert json_parse('{"age": 21, "state":"CO","occupation":"rides the rodeo"}') == (
                      ['value', ['object', '{', ['members', ['pair', ['string', '"age"'], 
                       ':', ['value', ['number', ['int', '21']]]], ',', ['members', 
                      ['pair', ['string', '"state"'], ':', ['value', ['string', '"CO"']]], 
                      ',', ['members', ['pair', ['string', '"occupation"'], ':', 
                      ['value', ['string', '"rides the rodeo"']]]]]], '}']], '')
    return 'tests pass'

print(test())

tests pass


/Users/yangzhan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: FutureWarning: Possible nested set at position 5


## 3.2 inverse function 反函数

In [53]:
# --------------
# User Instructions
#
# Write a function, inverse, which takes as input a monotonically
# increasing (always increasing) function that is defined on the 
# non-negative numbers. The runtime of your program should be 
# proportional to the LOGARITHM of the input. You may want to 
# do some research into binary search and Newton's method to 
# help you out.
#
# This function should return another function which computes the
# inverse of the input function. 
#
# Your inverse function should also take an optional parameter, 
# delta, as input so that the computed value of the inverse will
# be within delta of the true value.

# -------------
# Grading Notes
#
# Your function will be called with three test cases. The 
# input numbers will be large enough that your submission
# will only terminate in the allotted time if it is 
# efficient enough. 

def slow_inverse(f, delta=1/128.):
    """Given a function y = f(x) that is a monotonically increasing function on
    non-negatve numbers, return the function x = f_1(y) that is an approximate
    inverse, picking the closest value to the inverse, within delta."""
    def f_1(y):
        x = 0
        while f(x) < y:
            x += delta
        # Now x is too big, x-delta is too small; pick the closest to y
        return x if (f(x)-y < y-f(x-delta)) else x-delta
    return f_1 

def inverse(f, delta = 1/128.):
    """Given a function y = f(x) that is a monotonically increasing function on
    non-negatve numbers, return the function x = f_1(y) that is an approximate
    inverse, picking the closest value to the inverse, within delta."""
    def fn(y):
        high = float(y)
        low = 0.0
        i = 0
        while True:
            i += 1
            x = ( high + low )/2.
            if abs( f( x ) - y ) < y * 10 **( -12)  or i > 399 :
                return float(x)
            elif f(x) < y:
                low = x + 1.0
            elif f(x) > y:
                high = x - 1.0
    return fn
    

cuberoot = slow_inverse(lambda x:x**9)
cuberoot1 = inverse(lambda x:x**9)



print(cuberoot(1e9))
print(cuberoot1(1e9))

10.0
9.999999999999567


## 3.3 find html tags

In [56]:
# ---------------
# User Instructions
#
# Write a function, findtags(text), that takes a string of text
# as input and returns a list of all the html start tags in the 
# text. It may be helpful to use regular expressions to solve
# this problem.

import re

def findtags(text):
    # your code here
#     params = '(\w+\s*=\s*"[^"]*"\s*)*'
#     tags = '(<\s*\w+\s*'+params+'\s*/?>)'
#     return re.findall(tags,text)
  
    p = re.compile('< *[a-zA-Z]+ *(?: *[a-z_\-A-Z]+(?:="[^"]*")?)* *>')
    return  p.findall(text)



testtext1 = """
My favorite website in the world is probably 
<a href="www.udacity.com">Udacity</a>. If you want 
that link to open in a <b>new tab</b> by default, you should
write <a href="www.udacity.com"target="_blank">Udacity</a>
instead!
"""

testtext2 = """
Okay, so you passed the first test case. <let's see> how you 
handle this one. Did you know that 2 < 3 should return True? 
So should 3 > 2. But 2 > 3 is always False.
"""

testtext3 = """
It's not common, but we can put a LOT of whitespace into 
our HTML tags. For example, we can make something bold by
doing <         b           > this <   /b    >, Though I 
don't know why you would ever want to.
"""

def test():
    print(findtags(testtext1))
    assert findtags(testtext1) == ['<a href="www.udacity.com">', 
                                   '<b>', 
                                   '<a href="www.udacity.com"target="_blank">']
    assert findtags(testtext2) == []
    assert findtags(testtext3) == ['<         b           >']
    return 'tests pass'

print(test())

['<a href="www.udacity.com">', '<b>', '<a href="www.udacity.com"target="_blank">']
tests pass
